```
pip install ipywidgets

pip install ipyleaflet
jupyter nbextension enable --py widgetsnbextension

pip install bqplot
jupyter nbextension enable --py --sys-prefix bqplot
```

In [1]:
# core
import os
import json
import requests
import numpy as np
import pandas as pd
from osgeo import gdal,ogr,osr
from math import ceil

# widgets
import ipywidgets as wg
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles, GeoJSON
import matplotlib.pyplot as plt

# some constants
basemap = basemap_to_tiles(basemaps.Esri.WorldImagery)
smap_res = 9
layout = {'border':'0.5px solid gray','overflow_y':"auto",'width':'100%','height':'400px'}

### Open shapefile; make widgets; def callback function for map interaction

1. set driver; open sites shapefile with ogr; 
2. init widgets
3. define callback function

In [2]:
# -------------------------------------------------------------------------------
# 1 open sites shapefile

# get driver; open shapefile; get extent; get centroid
driver = ogr.GetDriverByName("ESRI Shapefile")
sites = driver.Open("sites/Sites_lf.shp", 0)
xmin, xmax, ymin, ymax = sites[0].GetExtent()
centroid = ((ymax+ymin)/2, (xmax+xmin)/2)

# get transform to wgs84
source = sites[0].GetSpatialRef()
target = osr.SpatialReference()
target.ImportFromEPSG(4326)
transform = osr.CoordinateTransformation(source, target)

# transform centroid
centroid_wkt = "POINT ({x} {y})".format(x=centroid[1], y=centroid[0])
centroid_geom = ogr.CreateGeometryFromWkt(centroid_wkt)
centroid_geom.Transform(transform)
centroid_wgs84 = (centroid_geom.GetY(), centroid_geom.GetX())

# -------------------------------------------------------------------------------
# 2 init map and other widgets

m = Map(layers=(basemap,), center=centroid_wgs84, zoom=4)           # map

output = wg.Output(layout=layout)                                   # meta box
with output:
    print("Click a polygon on the map to display metadata.")

ui = wg.HBox([wg.HBox([output], layout=wg.Layout(width="40%")), m]) # ui 

# -------------------------------------------------------------------------------
# 3 callback function

def callback1(**kwargs):

    if 'id' in kwargs.keys():
        
        # get the geojson for the selected map feature
        fid = kwargs['id'] 
        properties = kwargs['properties'] 
        site = sitesdata[fid]
        
        # get native projection feature extent; get number of samples in x,y
        geom = site['feature']
        xmin, xmax, ymin, ymax = geom.GetGeometryRef().GetEnvelope()
        nxsamples = ceil((xmax-xmin)/smap_res)
        nysamples = ceil((ymax-ymin)/smap_res)
        
        # generate array of samples to submit to soil moisture viz
        xsamples = np.array([xmin]+[xmin+i*smap_res for i in range(0,nxsamples)])
        ysamples = np.array([ymin]+[ymin+i*smap_res for i in range(0,nysamples)])
        
        # <grid sampling goes here ->
        """
        some numpy code to generate coordinate arrays that match
        the smap grid. i think i have some to plug in here already
        """
        # <- grid sampling goes here>
                
        # print some metadata to the display window
        output.clear_output() 
        with output:  
            for k,v in site['details'].items(): 
                print(k+":"+str(v))
                
            df = sitesdata[fid]['df']
            print(df)

### Loop over shapefile features (sites) and collect some key info in eight steps:

1. get geojson geometry of site boundary; get the attribute fields (called properties in geojson)
2. make ANPP pandas data frame for site
3. get the remaining site details
4. add to dictionary
5. add geojson to map

In [3]:
# dictionary for site data
sitesdata = {}

# loop over sites
for site in sites[0]:
    
    # ---------------------------------------------------------------------------
    # 1 reproject; get geojson features in native projection and wgs84
    
    # get feature properties (equivalent to attributes in arcmap)
    geojson = json.loads(site.ExportToJson())
    fid = geojson['id']
    properties = geojson['properties']
    
    # transform site feature to wgs84; make new geojson feature with properties
    geom = site.GetGeometryRef()
    geom.Transform(transform)
    geojson_wgs84 = {'type': 'Feature', 
                     'geometry': json.loads(geom.ExportToJson()),
                     'properties': properties,
                     'id': fid}
    
    # ---------------------------------------------------------------------------
    # 2 get length of series; make ANPP data frame for mean, std
    
    nyears = len([key for key in properties.keys() if "MEAN" in key])
    data = {'year': [], 'mean': [], 'std': []}
    for i in range(1,nyears):
        data['year'].append(1980+i)
        data['mean'].append(properties["MEAN_"+str(i)])
        data['std'].append(properties["STD_"+str(i)])
    df = pd.DataFrame(data)
    
    # ---------------------------------------------------------------------------
    # 3 get remaining site information
    
    details = {}
    for prop, value in properties.items():
        if all(["Count" not in prop, 
                "MEAN" not in prop, 
                "STD" not in prop]):
            details[prop] = value

    # ---------------------------------------------------------------------------        
    # 4 insert to sites dictionary
    
    sitesdata[fid] = {
        'geojson': geojson,
        'feature': site,
        'properties': properties,
        'df': df,
        'details': details}
        
    # ---------------------------------------------------------------------------    
    # 5 add wgs84 geojson to map
    
    poly = GeoJSON(data=geojson_wgs84)
    poly.on_click(callback1)
    m.add_layer(poly)

### call widgets -->> run this cell to displau map. click poly to display meta and anpp series

In [4]:
ui